In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the data
file_path = '/Users/liangying/Desktop/航港數據競賽/兩岸TEU數:噸數:艘次/8.TEU進出轉口合併.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Convert the "年月" column to datetime format and set it as the index
data['年月'] = pd.to_datetime(data['年月'])
data.set_index('年月', inplace=True)

# Filter the data for '進口'
import_data = data[data['進出轉口'] == '進口']

# Aggregate the TEU values by month
monthly_import_data = import_data.resample('M').sum()

# Decompose the time series
result = seasonal_decompose(monthly_import_data['TEU'], model='additive')

# Plotting the decomposition
plt.figure(figsize=(12, 10))
plt.subplot(411)
plt.plot(result.observed, label='Observed')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(result.trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(result.seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(result.resid, label='Residual')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

# Fit the Holt-Winters model to the data
hw_model = ExponentialSmoothing(
    monthly_import_data['TEU'],
    trend='additive',
    seasonal='additive',
    seasonal_periods=12
).fit()

# Forecast future values (e.g., next 12 months)
hw_forecast = hw_model.forecast(steps=12)

# Plot the historical data and the forecast
plt.figure(figsize=(10, 6))
plt.plot(monthly_import_data['TEU'], label='Historical TEU')
plt.plot(hw_forecast, label='Forecasted TEU', linestyle='--')
plt.xlabel('Date')
plt.ylabel('TEU')
plt.title('TEU Import Forecast using Holt-Winters Exponential Smoothing')
plt.legend()
plt.show()

# Calculate performance metrics for Holt-Winters
hw_forecast_train = hw_model.fittedvalues
hw_mae = mean_absolute_error(monthly_import_data['TEU'], hw_forecast_train)
hw_mse = mean_squared_error(monthly_import_data['TEU'], hw_forecast_train)
hw_rmse = np.sqrt(hw_mse)

print('Holt-Winters MAE:', hw_mae)
print('Holt-Winters MSE:', hw_mse)
print('Holt-Winters RMSE:', hw_rmse)

# Find the best parameters for ARIMA using grid search
best_aic = float('inf')
best_order = None
best_model = None

for p in range(6):
    for q in range(6):
        try:
            model = ARIMA(monthly_import_data['TEU'], order=(p, 1, q))
            model_fit = model.fit()
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_order = (p, 1, q)
                best_model = model_fit
        except:
            continue

# Fit the ARIMA model with the best parameters
model = ARIMA(monthly_import_data['TEU'], order=best_order)
model_fit = model.fit()

# Forecast future values (e.g., next 12 months)
arima_forecast = model_fit.forecast(steps=12)

# Plot the historical data and the forecast
plt.figure(figsize=(10, 6))
plt.plot(monthly_import_data['TEU'], label='Historical TEU')
plt.plot(arima_forecast, label='Forecasted TEU', linestyle='--')
plt.xlabel('Date')
plt.ylabel('TEU')
plt.title('TEU Import Forecast using ARIMA')
plt.legend()
plt.show()

# Calculate performance metrics for ARIMA
arima_forecast_train = model_fit.fittedvalues
arima_mae = mean_absolute_error(monthly_import_data['TEU'], arima_forecast_train)
arima_mse = mean_squared_error(monthly_import_data['TEU'], arima_forecast_train)
arima_rmse = np.sqrt(arima_mse)

print('ARIMA MAE:', arima_mae)
print('ARIMA MSE:', arima_mse)
print('ARIMA RMSE:', arima_rmse)
print('Best ARIMA Order:', best_order)


ModuleNotFoundError: No module named 'matplotlib'